In [1]:
import Analysis
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import regression_cv as rcv
%matplotlib

import copy
import FitRoutine2 as fr

Using matplotlib backend: TkAgg


In [2]:
%matplotlib

Using matplotlib backend: TkAgg


In [13]:
bf = np.load('2014bf_new_prune_edit.npy')

In [ ]:
run = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))

In [3]:
run = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_new.npy'), Analysis.bval2011, np.load('2014bf_new_prune_edit.npy'))

# All Fits

In [ ]:
n = 0

for i in run.prune_data:
    f = 0
    for j in i.fits:
        fig = plt.figure()
        i.plot()
        j.plot(False,linewidth=2)
        plt.xlim(min(i.x),max(i.x))
        plt.title('File {}, fit {}'.format(n,f))
        f+=1
    n+=1

In [ ]:
m = 0
for i in run.prune_data:
    if len(i.fits) > m:
        m = len(i.fits)

L = len(run.prune_data)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig = plt.figure(figsize=(20,70))
for i in run.prune_data:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([min(i.x),max(i.x)])

        c += 1
        temp += 1
    c += (m-temp)
    n += 1
fig.tight_layout()

In [ ]:
L = 0
for i in run.prune_data:
    for j in i.fits:
        L += 1
        
c = 1
fig = plt.figure(figsize=(5,100))
for i in run.prune_data:
    for j in i.fits:
        ax = plt.subplot(L,1,c)
        i.plot()
        j.plot(False,linewidth=1)
        #ax.set_xlim(j.pos_left,j.pos_right) #Tight
        ax.set_xlim(min(i.x),max(i.x)) #Full
        (y_min,y_max) = ax.get_ylim()
        dy = (y_max - y_min)*0.1
        ax.set_ylim(y_min - dy, y_max + dy)
        ax.tick_params(axis='both', which = 'both', bottom = 'off', top = 'off', left = 'off', right = 'off', labelleft = 'off', labelbottom = 'off')
        c += 1
plt.show()

In [ ]:
fig.savefig('2011_new_prune_all_fits',dpi=1000)

# Checking Regressions

In [4]:
def search(seq, t):  
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max <= min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m


def plot(data,result,full=False,if_data=True):
    x_min = result['x_min']
    x_max = result['x_max']
    if full:
        x = data.x
        y = data.y
    else:
        i_min = search(data.x,x_min)
        i_max = search(data.x,x_max)+1

        x = data.x[i_min:i_max]
        y = data.y[i_min:i_max]

    v0 = (x_min+x_max)/2.0

    params = result['result'].x
    model_l = params[0] + params[1] * np.sin(x*params[2] + (params[3]+params[4]*np.pi)) + params[5] * (x - v0)
    model_r = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[5] * (x - v0)
    if if_data:
        plt.plot(x,y)
    plt.plot(x,model_r,c='r')
    plt.plot(x,model_l,c='k')

In [5]:
def csf(scipy_fit):
    x = scipy_fit['result'].x
    out = list()
    out.append(x[2])
    out.append(x[1])
    out.append(x[3]+x[4]*np.pi)
    out.append(x[3])
    out.append(x[0])
    out.append(x[5])
    out.append(scipy_fit['val_acc'])
    out.append(scipy_fit['x_min'])
    out.append(scipy_fit['x_max'])
    phi = x[4]
    out.append(phi)
    out.append(scipy_fit['ci_deltaphi'])
    out.append((scipy_fit['x_min'] + scipy_fit['x_max'])/2)
    return np.array(out)

In [ ]:
bf = list()

In [6]:
#reg45 = np.load('2011_CV_new_res_flat_d45.npy')
reg45 = np.load('2014_CV_new_res_d45.npy')

In [7]:
#reg5 = np.load('2011_CV_new_res_flat_d5.npy')
reg5 = np.load('2014_CV_new_res_d5.npy')

In [8]:
#reg6 = np.load('2011_CV_new_res_flat_d6.npy')
reg6 = np.load('2014_CV_new_res_d6.npy')

In [9]:
#reg7 = np.load('2011_CV_new_res_flat_d7.npy')
reg7 = np.load('2014_CV_new_res_d7.npy')

In [17]:
filenum = 9
val_acc = 0.70
ci_deltaphi = 0.01

In [ ]:
%matplotlib

In [ ]:
t = reg7[filenum]
count = 0
for i in t:
    if i['val_acc']>val_acc and 0.16 < i['result'].x[4] < 1.80:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(count) + ', '+ str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4])+ ', '+ str(i['val_acc']))
    count += 1

In [21]:
i = reg5[filenum][45]
plt.figure()
plot(run.prune_data[filenum],i,True)
plt.title(str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4])+ ', '+ str(i['val_acc']))

In [ ]:
index_hold = [34,34,34,34]
for j in range(len(index_hold)):    
    i = [reg7[filenum],reg6[filenum],reg5[filenum],reg45[filenum]][j][index_hold[j]]
    plt.figure()
    plot(run.prune_data[filenum],i)
    plt.title(str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4])+ ', '+ str(i['val_acc']))

In [22]:
t = reg6[filenum]
start = 40
length = 11
for i in t[start:start+length]:
    if i['val_acc']>0 and 0.16 < i['result'].x[4] < 1.80:
        plt.figure()
        plot(run.prune_data[filenum],i)
        plt.title(str(start) + ', '+ str(i['ci_deltaphi']) + ', '+ str(i['result'].x[4])+ ', '+ str(i['val_acc']))
    start += 1

In [3]:
%matplotlib inline

In [4]:
import ipywidgets

In [ ]:
filenum = 5

In [ ]:
#fit = run.prune_data[filenum].fits[0]
#fit.freq = run.prune_data[filenum].fits[-2].freq
#fit = Analysis.Fit(csf(reg5[filenum][45]))
fit = Analysis.Fit(temp)

def func(phase_l,phase_r):
    def helper(fit, phase_l, phase_r):
        fig = plt.figure(figsize=(20,7))
        run.prune_data[filenum].plot()
        xpos = np.linspace(-40, -4, 10000)
        ytargl = fit.height + fit.amp * np.sin(fit.freq * xpos + phase_l*np.pi) + fit.slope * (xpos - fit.v0)
        ytargr = fit.height + fit.amp * np.sin(fit.freq * xpos + phase_r*np.pi) + fit.slope * (xpos - fit.v0)
        plt.plot(xpos, ytargl, 'k', linewidth = 2)
        plt.plot(xpos, ytargr, 'r', linewidth = 2)
        plt.title((phase_l - phase_r)%2.0)
    helper(fit,phase_l,phase_r)


interactive_plot = ipywidgets.interactive(func, phase_l=(-2.0, 2.0, 0.01), phase_r=(-2.0, 2.0, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '500px'
output.layout.width = '2000px'
interactive_plot

In [6]:
fit = run.prune_data[7].fits[1]

def func(phase_l,phase_r,freq):
    def helper(fit, phase_l, phase_r):
        fig = plt.figure(figsize=(20,7))
        run.prune_data[7].plot()
        xpos = np.linspace(-40, -4, 10000)
        ytargl = fit.height + fit.amp * np.sin(freq * xpos + phase_l*np.pi) + fit.slope * (xpos - fit.v0)
        ytargr = fit.height + fit.amp * np.sin(freq * xpos + phase_r*np.pi) + fit.slope * (xpos - fit.v0)
        plt.plot(xpos, ytargl, 'k', linewidth = 2)
        plt.plot(xpos, ytargr, 'r', linewidth = 2)
        plt.title((phase_l - phase_r)%2.0)
    helper(fit,phase_l,phase_r)


interactive_plot = ipywidgets.interactive(func, phase_l=(-2.0, 2.0, 0.01), phase_r=(-2.0, 2.0, 0.01),freq=(0.5,0.8,0.01))
output = interactive_plot.children[-1]
output.layout.height = '500px'
output.layout.width = '2000px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description=u'phase_l', max=2.0, min=-2.0, step=0.01), FloatSlider(value=0.0, description=u'phase_r', max=2.0, min=-2.0, step=0.01), FloatSlider(value=0.65, description=u'freq', max=0.8, min=0.5, step=0.01), Output(layout=Layout(height=u'500px', width=u'2000px'))), _dom_classes=('widget-interact',))

In [ ]:
temp = csf(reg5[filenum][45])

In [ ]:
temp[0]

In [ ]:
np.save('2014_0_extrafit',temp)

In [ ]:
run.prune_data[filenum].plot_fit(0,False)


In [ ]:
run.prune_data[filenum].fits[1].delta_phi #= (run.prune_data[filenum].fits[1].phase_l - run.prune_data[filenum].fits[1].phase_r)/np.pi%2

In [ ]:
hold = list(bf[filenum])
hold.insert(3,csf(reg5[filenum][34]))
bf[filenum] = np.array(hold)

In [ ]:
bf[filenum][0][9] = 2 - bf[filenum][0][9]

In [ ]:
fit.output_array()

In [ ]:
for i in run.prune_data[filenum].fits:
    print(i.output_array()[9])

In [15]:
bf[filenum][3] = csf(reg7[filenum][0])

In [ ]:
bf[filenum][0][3] = -0.99*np.pi

In [16]:
np.save('2014bf_new_prune_edit',bf)

In [ ]:
filenum = 6

In [ ]:
hold = list()
hold.append(csf(reg45[filenum][12]))
#hold.append(csf(reg45[filenum][19]))
#hold.append(csf(reg5[filenum][10]))
bf.append(np.array(hold))

In [ ]:
print(len(bf))

In [ ]:
bf_final = list()
for i in bf:
    temp = list()
    for j in i:
        temp.append(csf(j))
    bf_final.append(np.array(temp))


In [ ]:
L = 0
for i in run.prune_data:
    L += len(i.fits)
        
n = 0
c = 1
fig = plt.figure(figsize=(5,200))
for i in run.prune_data:
    if len(i.fits) > 0:
        for j in i.fits:
            ax = plt.subplot(L,1,c)
            i.plot()
            c += 1
            j.plot(False,linewidth=1)
            #ax.set_xlim(j.pos_left,j.pos_right) #Tight
            ax.set_xlim(min(i.x),max(i.x)) #Full
            (y_min,y_max) = ax.get_ylim()
            dy = (y_max - y_min)*0.1
            ax.set_ylim(y_min - dy, y_max + dy)
            ax.tick_params(axis='both', which = 'both', bottom = 'off', top = 'off', left = 'off', right = 'off', labelleft = 'off', labelbottom = 'off')
    n += 1
plt.show()

In [ ]:
fig.savefig('2014_new_prune_new_reg',dpi=1200)

# Build SE

In [3]:
run = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_new.npy'), Analysis.bval2014, np.load('2014bf_new_prune_edit.npy'))

In [4]:
se = np.load('2014se_edit.npy')

In [5]:
mom = 5
mindiff = 0.03
dV = 0.1
minsep = 1.0

In [6]:
filenum = 8

In [7]:
x = run.prune_data[filenum].x
y = run.prune_data[filenum].y
fit_hold = run.prune_data[filenum].fits
b = run.prune_data[filenum].b_val

In [8]:
e = fr.findEdge(x,y,fit_hold,dV=dV,momentum=mom,mindiff = mindiff)

In [9]:
fs = fr.finalSlips(x,y,fit_hold,dV=dV,momentum=mom,minsep=minsep,mindiff = mindiff)

In [10]:
c = 0
for i in e:
    if len(i)>0:
        plt.figure()
        plt.plot(x,y)
        fit_hold[c].plot(False)
        for j in i:
            index = rcv.search(x,j[3])
            plt.plot(x[index],y[index],'c*',markersize=20)
            index = rcv.search(x,j[2])
            plt.plot(x[index],y[index],'y*',markersize=20)
            plt.title(j[0])
    c += 1

In [11]:
for k in fs:
    plt.figure()
    plt.plot(x,y)
    for i in k:
        j = (i[2]+i[3])/2.0
        index = rcv.search(x,j)
        plt.plot(x[index],y[index],'y*',markersize=20)
        plt.title(i[0])

In [12]:
temp = list(fr.countEdge(x,y,fit_hold,b,dV=dV,momentum=mom,minsep=minsep,mindiff = mindiff))

In [13]:
temp = list(se[filenum])

In [20]:
temp

[array([-30.11417   ,   5.5805    ,   1.64553771,   1.        ]),
 array([-28.8464    ,   5.5805    ,   1.64553771,   1.        ]),
 array([-26.6193    ,   5.5805    ,   1.64553771,   1.        ]),
 array([-27.5842    ,   5.5805    ,   1.64553771,   1.        ]),
 array([-6.981628  ,  5.5805    ,  1.25720481,  1.        ]),
 array([-6.43602   ,  5.5805    ,  1.25720481,  1.        ]),
 array([-6.603667  ,  5.5805    ,  1.25720481,  1.        ]),
 array([-13.8221   ,   5.5805   ,   1.0183376,   1.       ]),
 array([-25.1515    ,   5.5805    ,   1.74004121,   1.        ]),
 array([-25.007     ,   5.5805    ,   1.74004121,   1.        ]),
 array([-21.11237   ,   5.5805    ,   1.74004121,   1.        ]),
 array([-19.5402    ,   5.5805    ,   1.74004121,   1.        ]),
 array([-19.18      ,   5.5805    ,   1.74004121,   1.        ]),
 array([-19.065     ,   5.5805    ,   1.74004121,   1.        ]),
 array([-18.616985  ,   5.5805    ,   1.74004121,   1.        ])]

In [21]:
plt.figure()
plt.plot(x,y)
for i in temp:
    j = i[0]
    index = rcv.search(x,j)
    plt.plot(x[index],y[index],'y*',markersize=20)

In [30]:
del temp[2]

In [16]:
temp.insert(8,np.array(temp[8]))

In [19]:
temp[9][0] = -25.007

In [492]:
se.append(np.array(temp))

In [493]:
len(se)

15

In [491]:
temp = np.array([])

In [22]:
se[filenum] = np.array(temp)

In [23]:
np.save('2014se_edit',se)

In [204]:
run = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_strong.npy'), Analysis.bval2014, np.load('2014bf_new_prune_edit.npy'))

In [209]:
run.prune_data[3].plot_fit(2)

In [32]:
se[filenum] = np.array(temp)

# Plot all edges, rethink dV

In [88]:
#Plot with histogram and edges:

run = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))
bf = np.load('2011bf_new_prune_flat_edit.npy')
se = np.load('2011se_flat_edit.npy')

m = 0
for i in bf:
    if len(i) > m:
        m = len(i)
m += 1 #for histo


L = len(bf)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig1 = plt.figure(figsize=(20,40))
for i in run.prune_data[0:L/2]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([j.pos_left,j.pos_right])
        if len(se[n]) > 0:
            for k in se[n][np.abs(se[n][:,2] - bf[n][temp,9])<0.01]:
                ax.axvline(k[0],color='k',lw=1)
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    if len(se[n]) > 0:
        plt.hist(se[n][:,2], bins=bins)
        plt.xlim([0,2.0])
        plt.xticks([1.0,1.25,1.5,1.75])
    c += (m-temp)
    n += 1

fig2 = plt.figure(figsize=(20,40))
c = 1
for i in run.prune_data[L/2:]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([j.pos_left,j.pos_right])
        if len(se[n]) > 0:
            for k in se[n][np.abs(se[n][:,2] - bf[n][temp,9])<0.01]:
                ax.axvline(k[0],color='k',lw=1)
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    if len(se[n]) > 0:
        plt.hist(se[n][:,2], bins=bins)
        plt.xlim([0,2.0])
        plt.xticks([1.0,1.25,1.5,1.75])
    c += (m-temp)
    n += 1
#fig.tight_layout()

In [89]:
fig1.savefig('2011 All Fits and Slips 1_2',dpi=1000)
fig2.savefig('2011 All Fits and Slips 2_2',dpi=1000)

In [34]:
#Plot with histogram and edges:

run = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_new.npy'), Analysis.bval2014, np.load('2014bf_new_prune_edit.npy'))
bf = np.load('2014bf_new_prune_edit.npy')
se = np.load('2014se_edit.npy')

m = 0
for i in bf:
    if len(i) > m:
        m = len(i)
m += 1 #for histo


L = len(bf)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig1 = plt.figure(figsize=(20,40))
for i in run.prune_data[0:L/2]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([j.pos_left,j.pos_right])
        if len(se[n]) > 0:
            for k in se[n][np.abs(se[n][:,2] - bf[n][temp,9])<0.01]:
                ax.axvline(k[0],color='k',lw=1)
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    if len(se[n]) > 0:
        plt.hist(se[n][:,2], bins=bins)
        plt.xlim([0,2.0])
        plt.xticks([1.0,1.25,1.5,1.75])
    c += (m-temp)
    n += 1
fig1.tight_layout()
fig2 = plt.figure(figsize=(20,40))
c = 1
for i in run.prune_data[L/2:14]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([j.pos_left,j.pos_right])
        if len(se[n]) > 0:
            for k in se[n][np.abs(se[n][:,2] - bf[n][temp,9])<0.01]:
                ax.axvline(k[0],color='k',lw=1)
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    if len(se[n]) > 0:
        plt.hist(se[n][:,2], bins=bins)
        plt.xlim([0,2.0])
        plt.xticks([1.0,1.25,1.5,1.75])
    c += (m-temp)
    n += 1
fig2.tight_layout()

In [35]:
fig1.savefig('2014 All Fits and Slips 1_2',dpi=1000)
fig2.savefig('2014 All Fits and Slips 2_2',dpi=1000)

In [ ]:
#Plot without histogram and edges:

run = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))

m = 0
for i in run.prune_data:
    if len(i.fits) > m:
        m = len(i.fits)
m +=1 #For all fits on one
        
L = len(run.prune_data)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig = plt.figure(figsize=(10,20))
for i in run.prune_data:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([min(i.x),max(i.x)])
        c += 1
        temp += 1
    ax = plt.subplot(L,m,c)
    i.plot()
    plt.xlim([min(i.x),max(i.x)])
    for j in i.fits:
        j.plot(False)
    c += 1
    temp += 1
    c += (m-temp)
    n += 1
#fig.tight_layout()

In [ ]:
fig.savefig('2011 New Prune Updated Slips Flat',dpi=1000)

In [ ]:
l = [[0,0],[1,0],[2,0],[3,0],[7,0],[10,0],[10,2],[12,0],[13,0],[13,1],[14,1],[16,0],[16,1],[17,0],[18,0],[19,1],[20,0],[20,1],[21,0]]

In [ ]:
run = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat.npy'))


n = 0
plt.rc('font', size=6)    

fig, ax = plt.subplots(len(l), figsize=(0.5,100))
for i in l:
    plt.sca(ax[n])
    run.prune_data[i[0]].plot_fit(i[1],False)
    plt.xlim(run.prune_data[i[0]].fits[i[1]].pos_left,run.prune_data[i[0]].fits[i[1]].pos_right)
    n+=1
plt.tight_layout()

In [ ]:
fig.savefig('2011 rejected fits',dpi=1200)

In [ ]:
l = [[1,0,37],[3,1,36],[4,3,32],[20,0,11]]

In [ ]:
L = len(l)
n = 1
plt.rc('font', size=6)    

fig = plt.figure(figsize=(1,10))

for j in l:
    ax = plt.subplot(L,1,n)
    filenum = j[0]
    i = [reg7[filenum],reg6[filenum],reg5[filenum],reg45[filenum]][j[1]][j[2]]
    plot(run.prune_data[filenum],i)
    plt.title('$\Delta \phi$ = ' + str(i['result'].x[4]))
    n+=1
    

In [ ]:
fig.savefig('2011 missing fits',dpi=1200)

In [ ]:
#Plot without histogram and edges, as halves:

run = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))

m = 0
for i in run.prune_data:
    if len(i.fits) > m:
        m = len(i.fits)
m +=1 #For all fits on one
        
L = len(run.prune_data)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig1 = plt.figure(figsize=(20,20))
for i in run.prune_data[:L/2]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([min(i.x),max(i.x)])
        plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    i.plot()
    plt.xlim([min(i.x),max(i.x)])
    plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
    for j in i.fits:
        j.plot(False)
    c += 1
    temp += 1
    c += (m-temp)
    n += 1
fig1.tight_layout()

L = len(run.prune_data)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig2 = plt.figure(figsize=(20,20))
for i in run.prune_data[L/2:]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim([min(i.x),max(i.x)])
        plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    i.plot()
    plt.xlim([min(i.x),max(i.x)])
    plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
    for j in i.fits:
        j.plot(False)
    c += 1
    temp += 1
    c += (m-temp)
    n += 1
fig2.tight_layout()

In [ ]:
fig1.savefig('2011 New Prune Updated Slips Flat 1_2 edited',dpi=1000)
fig2.savefig('2011 New Prune Updated Slips Flat 2_2 edited',dpi=1000)

In [ ]:
#Plot without histogram and edges, as halves, fits only:

run = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))

m = 0
for i in run.prune_data:
    if len(i.fits) > m:
        m = len(i.fits)
m +=1 #For all fits on one
        
L = len(run.prune_data)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig1 = plt.figure(figsize=(20,20))
for i in run.prune_data[:L/2]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim(j.pos_left,j.pos_right)
        plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    i.plot()
    plt.xlim([min(i.x),max(i.x)])
    plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
    for j in i.fits:
        j.plot(False)
    c += 1
    temp += 1
    c += (m-temp)
    n += 1
fig1.tight_layout()

L = len(run.prune_data)
total = L*m
c = 1
n = 0
plt.rc('font', size=6)    
bins = np.arange(0.0,2.03,0.03)
fig2 = plt.figure(figsize=(20,20))
for i in run.prune_data[L/2:]:
    temp = 0
    for j in i.fits:
        ax = plt.subplot(L/2,m,c)
        i.plot()
        j.plot(False)
        plt.xlim(j.pos_left,j.pos_right)
        plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
        c += 1
        temp += 1
    ax = plt.subplot(L/2,m,c)
    i.plot()
    plt.xlim([min(i.x),max(i.x)])
    plt.ylim([min(i.y)-0.01,max(i.y)+0.01])
    for j in i.fits:
        j.plot(False)
    c += 1
    temp += 1
    c += (m-temp)
    n += 1
fig2.tight_layout()

In [ ]:
fig1.savefig('2011 New Prune Updated Slips Flat Windowed 1_2 edited',dpi=1000)
fig2.savefig('2011 New Prune Updated Slips Flat Windowed 2_2 edited',dpi=1000)